### 파이썬 버전 확인

In [ ]:
import sys
print(sys.version)

3.11.14 (main, Oct 21 2025, 18:27:30) [Clang 20.1.8 ]


## 네이버 쇼핑 리뷰를 보고 판매자 답변 자동 생성하기(정중/사과/보상 제안)

#### 데이터 출처 : https://github.com/bab2min/corpus

1. 라이브러리 설치 (Colab GPU / Python 3.11)

In [15]:
# Colab은 보통 torch 2.6.0+cu124가 이미 설치되어 있습니다.
# 설치가 안 되어 있거나 버전을 맞추고 싶다면 주석 해제하여 실행하세요.
# --- 맥북(Apple Silicon, CPU/MPS) ---
#!pip install -q -U pip
#!pip install -q "torch>=2.2" "torchvision>=0.17" "torchaudio>=2.2" \
#   "transformers>=4.41" "datasets>=2.19" accelerate evaluate \
#    "scikit-learn>=1.2,<1.7" tqdm peft

# --- Colab GPU(T4 등, CUDA 12.4) ---
# Colab에 기본 설치된 torch 2.6.0+cu124가 있으면 이 블록은 생략해도 됩니다.
# 없거나 버전을 맞추고 싶을 때만 주석 해제해서 실행하세요.
# !pip install -q --index-url https://download.pytorch.org/whl/cu124 \
#     torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124
# 공통 패키지 설치
!pip install -q -U "transformers>=4.41" "datasets>=2.19" accelerate evaluate \
    "scikit-learn>=1.2,<1.7" tqdm peft

2. 데이터 파싱

In [16]:
from pathlib import Path
import random

RAW_PATH = Path("./naver_shopping.txt")

def load_reviews(min_len=5, max_n=None, seed=42):
    random.seed(seed)
    rows = []
    with RAW_PATH.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t", 1)
            if len(parts) != 2:
                continue
            rating_s, text = parts
            try:
                rating = int(rating_s)
            except:
                continue
            text = text.strip()
            if len(text) < min_len:
                continue
            rows.append((rating, text))
    random.shuffle(rows)
    if max_n:
        rows = rows[:max_n]
    return rows

rows = load_reviews(max_n=30000)  # 8GB면 처음엔 3만 이하 추천
print("loaded:", len(rows))
print(rows[0])

loaded: 30000
(2, '냄새랑 맛이 생각보다 너무 역해요')


3. 불만 유형(키워드) 분류

In [17]:
COMPLAINT_RULES = {
    "배송": ["배송", "늦", "지연", "도착", "택배"],
    "품질/불량": ["불량", "하자", "고장", "깨", "찢", "누수", "작동", "불안정"],
    "포장": ["포장", "박스", "파손", "훼손", "완충"],
    "가격/가성비": ["비싸", "가격", "가성비", "싸", "할인"],
    "응대/서비스": ["응대", "문의", "연락", "고객센터", "불친절"],
}

def detect_topics(text: str):
    topics = []
    for topic, kws in COMPLAINT_RULES.items():
        if any(kw in text for kw in kws):
            topics.append(topic)
    return topics[:2]  # 너무 많으면 1~2개만

4. 템플릿 기반 판매자 답변 생성

In [18]:
def make_seller_reply(rating: int, review: str) -> str:
    topics = detect_topics(review)
    topic_phrase = " / ".join(topics) if topics else None

    # 보상 제안(너무 구체적 금액은 피하고 옵션만 제시)
    compensation = random.choice([
        "교환 또는 환불을 도와드리겠습니다.",
        "확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.",
        "불편을 줄이기 위해 교환/환불 절차를 빠르게 진행해드리겠습니다."
    ])

    # 긍정(4~5)
    if rating >= 4:
        extra = "앞으로도 더 좋은 상품과 서비스로 보답하겠습니다."
        upsell = random.choice([
            "재구매해주시면 감사하겠습니다.",
            "다음에도 만족스러운 경험을 드리겠습니다.",
            "소중한 후기 감사합니다."
        ])
        if topic_phrase:
            return f"안녕하세요, 고객님. {topic_phrase} 관련하여 만족하셨다니 정말 기쁩니다. {extra} {upsell}"
        return f"안녕하세요, 고객님. 소중한 후기 감사합니다. {extra} {upsell}"

    # 부정(1~2)
    if rating <= 2:
        apology = "불편을 드려 진심으로 죄송합니다."
        ask = "주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다."
        if topic_phrase:
            return f"안녕하세요, 고객님. {topic_phrase} 관련하여 {apology} {ask} {compensation}"
        return f"안녕하세요, 고객님. {apology} {ask} {compensation}"

    # 중립(3) — 과제에서는 제외해도 되고, 남겨도 됨
    neutral = "의견 남겨주셔서 감사합니다."
    improve = "말씀해주신 부분은 개선하여 더 나은 서비스로 보답하겠습니다."
    if topic_phrase:
        return f"안녕하세요, 고객님. {topic_phrase} 관련하여 {neutral} {improve}"
    return f"안녕하세요, 고객님. {neutral} {improve}"

# 샘플 확인
for r, t in rows[:5]:
    print("RATING:", r)
    print("REVIEW:", t)
    print("REPLY:", make_seller_reply(r, t))
    print("-"*80)

RATING: 2
REVIEW: 냄새랑 맛이 생각보다 너무 역해요
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.
--------------------------------------------------------------------------------
RATING: 5
REVIEW: 진짜대박좋아요만족^^
REPLY: 안녕하세요, 고객님. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 다음에도 만족스러운 경험을 드리겠습니다.
--------------------------------------------------------------------------------
RATING: 5
REVIEW: 좋아요 배송도 빠르고 가겨도 마니 저렴하고~~^^
REPLY: 안녕하세요, 고객님. 배송 관련하여 만족하셨다니 정말 기쁩니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 다음에도 만족스러운 경험을 드리겠습니다.
--------------------------------------------------------------------------------
RATING: 1
REVIEW: 1년 넘게 쓴 와이퍼보다 능력이 떨어지내요. 워셔액도 못 닦아내네요. 이런거 팔지 않으셨으면 좋겠네요
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.
--------------------------------------------------------------------------------
RATING: 2
REVIEW: 진짜뚱뚱하신분만사셔야될듯
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문

5. 학습 데이터 구성 + split

In [19]:
from sklearn.model_selection import train_test_split

def build_example(rating, review):
    # 프롬프트: 역할/규칙/입력
    prompt = (
        "### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n"
        "### 규칙:\n"
        "- 한국어 존댓말로 정중하게 작성\n"
        "- 2~4문장으로 간결하게\n"
        "- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함\n"
        "- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함\n"
        "### 고객 리뷰:\n"
        f"{review}\n"
        "### 판매자 답변:\n"
    )
    completion = make_seller_reply(rating, review)
    return {"prompt": prompt, "completion": completion, "rating": rating, "review": review}

# 3점은 빼서 “명확한” 긍/부정만 학습(추천)
filtered = [(r, t) for (r, t) in rows if r in (1,2,4,5)]
data = [build_example(r, t) for r, t in filtered[:20000]]  # 최대 2만개만

train_data, valid_data = train_test_split(data, test_size=0.05, random_state=42)

print(len(train_data), len(valid_data))
print(train_data[0]["prompt"])
print("->", train_data[0]["completion"])

19000 1000
### 역할: 당신은 온라인 쇼핑몰 판매자입니다.
### 규칙:
- 한국어 존댓말로 정중하게 작성
- 2~4문장으로 간결하게
- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함
- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함
### 고객 리뷰:
저렴하게 좋은 제품을 구매한거 같아 만족해요^^
### 판매자 답변:

-> 안녕하세요, 고객님. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 재구매해주시면 감사하겠습니다.


6. 모델/토크나이저 로드 + LoRA 적용

In [20]:
import torch, platform
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

MODEL_ID = "skt/kogpt2-base-v2"
def pick_device():
    # Colab GPU (CUDA)
    if torch.cuda.is_available():
        return "cuda"
    # M1/M2 Apple Silicon (MPS)
    if platform.system() == "Darwin" and torch.backends.mps.is_available():
        return "mps"
    # fallback CPU
    return "cpu"
device = pick_device()
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("mps available:", torch.backends.mps.is_available())
if torch.cuda.is_available():
    print("gpu name:", torch.cuda.get_device_name(0))
print("device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# GPT2류는 pad_token이 없는 경우가 많아서 eos로 맞추는 게 안전
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
model.config.use_cache = False
model.gradient_checkpointing_enable()  # 메모리 절약

# LoRA 설정 (GPT-2 계열에서 흔히 쓰는 target)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj"],  # GPT-2 계열에 흔한 모듈명
    fan_in_fan_out=True
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.to(device)

torch: 2.6.0+cu124
cuda available: True
mps available: False
gpu name: Tesla T4
device: cuda
trainable params: 811,008 || all params: 125,975,040 || trainable%: 0.6438


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(51200, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

7. Dataset 만들기 + 토크나이징

In [21]:
from datasets import Dataset

def pack_text(ex):
    return ex["prompt"] + ex["completion"]

train_ds = Dataset.from_dict({"text": [pack_text(x) for x in train_data]})
valid_ds = Dataset.from_dict({"text": [pack_text(x) for x in valid_data]})

MAX_LEN = 192  # 8GB 추천(256도 가능하지만 터질 수 있음)

def tok(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length"
    )

train_tok = train_ds.map(tok, batched=True, remove_columns=["text"])
valid_tok = valid_ds.map(tok, batched=True, remove_columns=["text"])


Map:   0%|          | 0/19000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

8. Trainer로 학습

In [22]:
import os, random, inspect, platform
import torch
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import get_peft_model, LoraConfig, TaskType

# 디바이스 선택 (CUDA 우선, 그다음 MPS, 아니면 CPU)
def pick_device():
    if torch.cuda.is_available():
        return "cuda"
    if platform.system() == "Darwin" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

device = pick_device()
print("torch =", torch.__version__)
print("cuda available =", torch.cuda.is_available())
print("mps available =", torch.backends.mps.is_available())
if torch.cuda.is_available():
    print("gpu name =", torch.cuda.get_device_name(0))
print("device =", device)

# 전제 체크
assert "tokenizer" in globals(), "tokenizer가 먼저 정의돼야 합니다."
assert "train_tok" in globals(), "train_tok(토큰화된 학습 데이터셋)이 먼저 정의돼야 합니다."
valid_tok = globals().get("valid_tok", None)
MODEL_ID = globals().get("MODEL_ID", "skt/kogpt2-base-v2")

# tokenizer / special token 안전 확인
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

print("len(tokenizer) =", len(tokenizer))
print("pad_token_id =", tokenizer.pad_token_id, "eos_token_id =", tokenizer.eos_token_id)

# 베이스 모델 로드 + 임베딩 리사이즈
base_model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
base_model.resize_token_embeddings(len(tokenizer))
base_model.config.pad_token_id = tokenizer.pad_token_id
if tokenizer.eos_token_id is not None:
    base_model.config.eos_token_id = tokenizer.eos_token_id
base_model.config.use_cache = False
base_model.gradient_checkpointing_enable()

# LoRA 설정
lora_config = globals().get("lora_config", None)
if lora_config is None:
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        target_modules=["c_attn"],
    )

model = get_peft_model(base_model, lora_config)

# 디바이스로 이동 (CUDA/MPS/CPU)
model.to(device)
try:
    model.print_trainable_parameters()
except Exception:
    pass

# 데이터 콜레이터
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# TrainingArguments
OUTPUT_DIR = "outputs/seller_reply_lora"
os.makedirs(OUTPUT_DIR, exist_ok=True)

common_kwargs = dict(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    max_steps=200,
    num_train_epochs=1,
    logging_steps=25,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
    fp16=(device == "cuda"),  # CUDA에서만 fp16, MPS/CPU는 float32
    seed=42,
)

try:
    args = TrainingArguments(
        **common_kwargs,
        eval_strategy="steps" if valid_tok is not None else "no",
        eval_steps=200 if valid_tok is not None else None,
    )
except TypeError:
    args = TrainingArguments(
        **common_kwargs,
        evaluation_strategy="steps" if valid_tok is not None else "no",
        eval_steps=200 if valid_tok is not None else None,
    )

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=valid_tok if valid_tok is not None else None,
    data_collator=collator,
)
if hasattr(trainer, "label_names"):
    trainer.label_names = ["labels"]

# 범위 체크
emb_size = model.get_input_embeddings().weight.shape[0]
idxs = random.sample(range(len(train_tok)), k=min(200, len(train_tok)))
mx = max(max(train_tok[i]["input_ids"]) for i in idxs)
print("model emb size =", emb_size, "| sample input_id max =", mx)
assert mx < emb_size, f"토큰 id({mx})가 임베딩 크기({emb_size}) 이상입니다. resize가 적용됐는지 확인하세요."

# 학습
train_result = trainer.train()

# 저장
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Done. Saved to:", OUTPUT_DIR)
print("Train result:", train_result)

torch = 2.6.0+cu124
cuda available = True
mps available = False
gpu name = Tesla T4
device = cuda
len(tokenizer) = 51201
pad_token_id = 51200 eos_token_id = 51200
trainable params: 811,008 || all params: 125,975,808 || trainable%: 0.6438
model emb size = 51201 | sample input_id max = 51200


Step,Training Loss,Validation Loss
200,0.691600,0.653286


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


✅ Done. Saved to: outputs/seller_reply_lora
Train result: TrainOutput(global_step=200, training_loss=1.230308208465576, metrics={'train_runtime': 335.354, 'train_samples_per_second': 9.542, 'train_steps_per_second': 0.596, 'total_flos': 316540138291200.0, 'train_loss': 1.230308208465576, 'epoch': 0.16842105263157894})


9. 생성 함수

In [48]:
import torch, platform, re
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from typing import Union, List, Dict, Any

# =========================
# Config
# =========================
MODEL_ID = "skt/kogpt2-base-v2"
CKPT_DIR = "outputs/seller_reply_lora"

# =========================
# Device picker
# =========================
def pick_device():
    if torch.cuda.is_available():
        return "cuda"
    if platform.system() == "Darwin" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

device = pick_device()
print("device =", device)

# =========================
# 1) ✅ Tokenizer: 베이스 모델에서 로드 (fast 사용)
# =========================
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
except Exception as e:
    print("AutoTokenizer(use_fast=True) failed ->", repr(e))
    raise

print("tokenizer class =", tokenizer.__class__.__name__)

# (안전) eos/pad 보장
# GPT2 계열은 pad_token이 없는 경우가 많아 새 pad 토큰을 추가하는 방식이 가장 안전함
if tokenizer.eos_token is None:
    # KoGPT2는 보통 eos가 있지만 혹시 없으면 지정
    tokenizer.eos_token = "<|endoftext|>"

if tokenizer.pad_token is None:
    # eos를 pad로 "대입"만 해도 되지만, 학습/패딩에서 더 안전하게 새 pad를 추가
    # (원래 vocab에 없는 토큰이면 길이가 +1 됨)
    tokenizer.add_special_tokens({"pad_token": "<pad>"})

print("len(tokenizer) =", len(tokenizer))
print("bos:", tokenizer.bos_token_id, "eos:", tokenizer.eos_token_id, "pad:", tokenizer.pad_token_id)

# =========================
# 2) Base model load
# =========================
base_model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

# tokenizer 길이에 맞춰 임베딩 리사이즈 (pad 추가했으면 +1 됨)
base_model.resize_token_embeddings(len(tokenizer))

base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.eos_token_id = tokenizer.eos_token_id

# =========================
# 3) Load LoRA adapter
# =========================
model = PeftModel.from_pretrained(base_model, CKPT_DIR)
model.to(device)
model.eval()

print("emb size       =", model.get_input_embeddings().weight.shape[0])
print("vocab==emb ?   =", len(tokenizer) == model.get_input_embeddings().weight.shape[0])

# ============================================================
# Debugging block A: tokenizer sanity
# ============================================================
print("=" * 80)
test_text = "리뷰: 배송이 늦어요\n판매자 답변:"
enc = tokenizer(test_text, add_special_tokens=True)
has_none = any(x is None for x in enc["input_ids"])
print("Tokenizer sanity: has None in input_ids? ->", has_none)

try:
    _ = tokenizer.decode(enc["input_ids"], skip_special_tokens=False)
    print("Tokenizer decode sanity: OK")
except Exception as e:
    print("Tokenizer decode sanity: FAIL ->", repr(e))

print("=" * 80)

# ============================================================
# 4) generate_reply (간단/안정형)
# ============================================================
def generate_reply(
    prompt: Union[str, List[str], Dict[str, Any]],
    max_new_tokens: int = 96,
    do_sample: bool = True,
    temperature: float = 0.8,
    top_p: float = 0.9,
    repetition_penalty: float = 1.08,
    debug: bool = False,
) -> str:
    # prompt -> str 정규화
    if isinstance(prompt, dict):
        prompt = prompt.get("prompt") or prompt.get("text") or prompt.get("review") or str(prompt)
    if isinstance(prompt, (list, tuple)):
        prompt = "\n".join(map(str, prompt))
    prompt = str(prompt)

    max_len = int(getattr(model.config, "n_positions", 1024) or 1024)

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_len,
        padding=False,
        add_special_tokens=True,
    )

    # 혹시라도 None이 끼면 여기서 즉시 알려줌
    if any(x is None for x in inputs["input_ids"].view(-1).tolist()):
        raise ValueError("Tokenizer produced None inside input_ids. Tokenizer 상태가 비정상입니다.")

    inputs = {k: v.to(device) for k, v in inputs.items()}

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=repetition_penalty,
    )
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))

    if debug:
        print("prompt head:", repr(prompt[:80]))
        print("pad/eos:", tokenizer.pad_token_id, tokenizer.eos_token_id)

    with torch.inference_mode():
        out = model.generate(**inputs, **gen_kwargs)

    full = tokenizer.decode(out[0], skip_special_tokens=True)
    # 프롬프트 제거
    return full[len(prompt):].strip() if full.startswith(prompt) else full.strip()

# ============================================================
# Debugging block B: quick generation test
# ============================================================
print("Quick generation test:")
try:
    print(generate_reply("리뷰: 배송이 늦어요\n판매자 답변:", max_new_tokens=60))
except Exception as e:
    print('this is error')
    print("generate_reply failed ->", repr(e))


device = cuda
tokenizer class = GPT2TokenizerFast
len(tokenizer) = 51201
bos: 51200 eos: 51200 pad: 3
emb size       = 51201
vocab==emb ?   = True
Tokenizer sanity: has None in input_ids? -> False
Tokenizer decode sanity: OK
Quick generation test:
안녕하세요, 고객님. 배송 관련하여 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 교환/환불 절차를 빠르게 진행하겠습니다. 확인 후 쿠폰/교환 중 하나를 택배로 보상해 드리겠습니다.


10. 프롬프트 3종 비교(단순/조건/강한 제약)

In [49]:
sample_review_neg = "배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다."
sample_review_pos = "배송도 빠르고 제품 품질이 좋아요. 재구매 의사 있습니다!"

def prompt_basic(review) -> str:
    return f"리뷰: {review}\n판매자 답변:"

def prompt_with_rules(review, neg=True) -> str:
    rules = (
        "규칙: 한국어 존댓말, 2~4문장.\n"
        + ("사과+해결책+보상(쿠폰/교환/환불 중 하나)을 반드시 포함.\n" if neg else "감사+재구매 유도 포함.\n")
    )
    return f"{rules}리뷰: {review}\n판매자 답변:"

def prompt_project_style(review: str) -> str:
    return (
        "### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n"
        "### 규칙:\n"
        "- 한국어 존댓말로 정중하게 작성\n"
        "- 2~4문장으로 간결하게\n"
        "- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함\n"
        "- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함\n"
        "### 고객 리뷰:\n"
        f"{review}\n"
        "### 판매자 답변:\n"
    )

for p in [prompt_basic(sample_review_neg),
          prompt_with_rules(sample_review_neg, neg=True),
          prompt_project_style(sample_review_neg)]:
    print(type(p), repr(p)[:120])
    print("PROMPT:\n", p)
    print("OUTPUT:\n", generate_reply(p))
    print("="*80)

for p in [prompt_basic(sample_review_pos),
          prompt_with_rules(sample_review_pos, neg=False),
          prompt_project_style(sample_review_pos)]:
    print("PROMPT:\n", p)
    print("OUTPUT:\n", generate_reply(p))
    print("="*80)


<class 'str'> '리뷰: 배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다.\n판매자 답변:'
PROMPT:
 리뷰: 배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다.
판매자 답변:
OUTPUT:
 안녕하세요, 고객님. 배송 관련하여 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 교환/환불 절차를 빠르게 진행하겠습니다.
안녕하세요, 고객님. 배송 관련하여 불편하신 사항에 대해 확인해드리겠습니다. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 재구매해주시면 감사하겠습니다. 소중한 후기 감사합니다. 다음에도 만족하겠습니다. 재
<class 'str'> '규칙: 한국어 존댓말, 2~4문장.\n사과+해결책+보상(쿠폰/교환/환불 중 하나)을 반드시 포함.\n리뷰: 배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다.\n판매자 답변:'
PROMPT:
 규칙: 한국어 존댓말, 2~4문장.
사과+해결책+보상(쿠폰/교환/환불 중 하나)을 반드시 포함.
리뷰: 배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다.
판매자 답변:
OUTPUT:
 안녕하세요, 고객님. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 교환/환불 관련하여 문의 남겨주시면 빠르게 도와드리겠습니다. 교환/환불 관련 사항 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다. 교환/환불 등 가능한 보상안을 안내드리겠습니다. 교환/환불 등 가능한 보상안을 안내드리겠습니다. 교환/환불 등의 가능한 보상안을 안내
<class 'str'> '### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n### 규칙:\n- 한국어 존댓말로 정중하게 작성\n- 2~4문장으로 간결하게\n- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함\n
PROMPT:
 ### 역할: 당신은 온라인 쇼핑몰 판매자입니다.
### 규칙:
-

11. 규칙 준수 점수(간단 룰 기반)

In [50]:
import re

def score_reply(review_rating, reply):
    score = 0
    # 문장 수(대충 .?! 기준)
    sentences = [s for s in re.split(r"[.?!]\s*", reply) if s.strip()]
    if 2 <= len(sentences) <= 4:
        score += 1

    if review_rating <= 2:
        if any(k in reply for k in ["죄송", "사과"]):
            score += 1
        if any(k in reply for k in ["교환", "환불", "쿠폰", "보상"]):
            score += 1
    if review_rating >= 4:
        if any(k in reply for k in ["감사", "고맙"]):
            score += 1
        if any(k in reply for k in ["재구매", "다음에도", "또 이용"]):
            score += 1
    return score

# 샘플 50개로 점수 보기
samples = random.sample(filtered, 50)
total = 0
for r, review in samples:
    p = prompt_project_style(review)
    out = generate_reply(p)
    total += score_reply(r, out)

print("avg rule score:", total/50)


avg rule score: 1.44
